In [1]:
import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.insert(0, parent_dir)

import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime

from CurveDataFetcher import CurveDataFetcher
from CurveBuilder import get_spot_rates_fitter, reprice_bonds

import nest_asyncio
nest_asyncio.apply()

%load_ext autoreload
%autoreload 2

In [2]:
curve_data_fetcher = CurveDataFetcher(use_ust_issue_date=True, debug_verbose=True)

In [13]:
as_of_date = datetime(2024, 8, 29)
quote_type = "bid"

curve_set_df = curve_data_fetcher.build_curve_set(
    as_of_date=as_of_date,
    sorted=True, 
    use_github=True, 
    # use_public_dotcom=True,
    include_off_the_run_number=True,
    market_cols_to_return=[f"{quote_type}_price", f"{quote_type}_yield"],
    # calc_free_float=True,
)
curve_set_df
# # remove OTRs and first off the runs due to liquidity premium
# filtered_curve_set_df = curve_set_df[(curve_set_df["rank"] != 0) & (curve_set_df["rank"] != 1)]

# # remove TBills
# filtered_curve_set_df = filtered_curve_set_df[
#     filtered_curve_set_df["security_type"] != "Bill"
# ]

# # remove low free float bonds (< $8bn)
# filtered_curve_set_df = filtered_curve_set_df[filtered_curve_set_df["free_float"] > 8000] 

# # filter bonds very close to maturity
# filtered_curve_set_df = filtered_curve_set_df[filtered_curve_set_df["time_to_maturity"] > 0.25] 

# # remove CTDs (this is a TODO)
# # filtered_curve_set_df = filtered_curve_set_df[filtered_curve_set_df["is_ctd"] == False] 

# filtered_curve_set_df

,cusip,security_type,auction_date,issue_date,maturity_date,time_to_maturity,int_rate,high_investment_rate,is_on_the_run,label,security_term,original_security_term,corpus_cusip,bid_price,bid_yield,rank
0,912797LA3,Bill,2024-08-01,2024-08-06,2024-09-03,0.013699,NaN,5.381000,False,"Sep 24s, 17-Week",4-Week,17-Week,NaN,99.898500,9.564397,16.0
1,912797LG0,Bill,2024-08-08,2024-08-13,2024-09-10,0.032877,NaN,5.381000,False,"Sep 24s, 17-Week",4-Week,17-Week,NaN,99.795444,6.969006,15.0
2,912797LH8,Bill,2024-08-15,2024-08-20,2024-09-17,0.052055,NaN,5.355000,False,"Sep 24s, 17-Week",4-Week,17-Week,NaN,99.693750,6.370058,14.0
3,912797LJ4,Bill,2024-08-22,2024-08-27,2024-09-24,0.071233,NaN,5.335000,True,"Sep 24s, 17-Week",4-Week,17-Week,NaN,99.592444,6.102740,13.0
4,912797LK1,Bill,2024-08-01,2024-08-06,2024-10-01,0.090411,NaN,5.346000,False,"Oct 24s, 17-Week",8-Week,17-Week,NaN,99.497361,5.847277,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388,912810TT5,Bond,2023-10-12,2023-10-16,2053-08-15,28.980822,4.125,NaN,False,"Aug 53s, 30-Year",29-Year 10-Month,30-Year,912803GU1,99.375000,4.162151,4.0
389,912810TV0,Bond,2024-01-11,2024-01-16,2053-11-15,29.232877,4.750,NaN,False,"Nov 53s, 30-Year",29-Year 10-Month,30-Year,912803GW7,110.187500,4.144628,3.0
390,912810TX6,Bond,2024-04-11,2024-04-15,2054-02-15,29.484932,4.250,NaN,False,"Feb 54s, 30-Year",29-Year 10-Month,30-Year,912803GY3,101.562500,4.157334,2.0
391,912810UA4,Bond,2024-07-11,2024-07-15,2054-05-15,29.728767,4.625,NaN,False,"May 54s, 30-Year",29-Year 10-Month,30-Year,912803HB2,108.156250,4.144435,1.0


In [42]:
t2 = np.linspace(1/365, 30, 1000)

# fitted_zero_curve_func_ns, status_ns = calibrate_ns_ols(
#     filtered_curve_set_df["time_to_maturity"].to_numpy(),
#     filtered_curve_set_df[f"{quote_type}_yield"].to_numpy(),
# )
# assert status_ns

# fitted_zero_curve_func_nss, status_nss, lstsq_res_nss = calibrate_nss_ols(
#     filtered_curve_set_df["time_to_maturity"].to_numpy(),
#     filtered_curve_set_df[f"{quote_type}_yield"].to_numpy(),
#     tau0=(1, 3.5)
# )
# assert status_nss

# print(status_nss)
# print(lstsq_res_nss)

# fitted_zero_curve_func_dl, status_dl = calibrate_diebold_li_ols(
#     filtered_curve_set_df["time_to_maturity"].to_numpy(),
#     filtered_curve_set_df[f"{quote_type}_yield"].to_numpy(),
# )

# fitted_zero_curve_func_bc, status_bc = calibrate_bc_ols(
#     np.concatenate((np.array([1/365]), filtered_curve_set_df["time_to_maturity"].to_numpy())),
#     np.concatenate((np.array([5.31]), filtered_curve_set_df[f"{quote_type}_yield"].to_numpy())),
# )

# fitted_zero_curve_func_aug_bc, status_aug_bc = calibrate_bc_augmented_ols(
#     np.concatenate((np.array([1/365]), filtered_curve_set_df["time_to_maturity"].to_numpy())),
#     np.concatenate((np.array([5.31]), filtered_curve_set_df[f"{quote_type}_yield"].to_numpy())),
# )

# filtered_curve_set_mles_func, status_mles = calibrate_mles_ols(
#     filtered_curve_set_df["time_to_maturity"].to_numpy(),
#     filtered_curve_set_df[f"{quote_type}_yield"].to_numpy(),
#     overnight_rate=5.31,
#     N=6
# )


fitted_zero_curve_dict = get_spot_rates_fitter(
    curve_set_df=filtered_curve_set_df,
    as_of_date=as_of_date,
    on_rate=5.35,
    # ql_fitting_methods=["ql_f_ns", "ql_f_nss", "ql_f_es", "ql_f_sp", "ql_f_cbs"],
    ql_fitting_methods=["ql_f_cbs"],
    ql_zero_curve_interp_method="ql_z_interp_monot_cubic",
    daily_interpolation=True,
    # simple_poly=3,
    knots=[-20, -10, -5, 0, 1, 2, 3, 5, 7, 10, 15, 20, 25, 30, 35, 40, 45, 50]
)

In [44]:
rv_df = reprice_bonds(
    as_of_date=as_of_date,
    ql_zero_curves={
        # "ns": fitted_zero_curve_dict["ql_f_ns"]["ql_zero_curve"],
        # "nss": fitted_zero_curve_dict["ql_f_nss"]["ql_zero_curve"],
        # "es": fitted_zero_curve_dict["ql_f_es"]["ql_zero_curve"],
        "cbs": fitted_zero_curve_dict["ql_f_cbs"]["ql_zero_curve"],
    },
    curve_set_df=curve_set_df,
)
rv_df

91282CFG1 degenerate single date (August 30th, 2024) schedule
 seed date: August 31st, 2024
 exit date: August 30th, 2024
 effective date: August 30th, 2024
 first date: null date
 next to last date: null date
 termination date: August 31st, 2024
 generation rule: Backward
 end of month: 0
912828YE4 degenerate single date (August 30th, 2024) schedule
 seed date: August 31st, 2024
 exit date: August 30th, 2024
 effective date: August 30th, 2024
 first date: null date
 next to last date: null date
 termination date: August 31st, 2024
 generation rule: Backward
 end of month: 0
9128282U3 degenerate single date (August 30th, 2024) schedule
 seed date: August 31st, 2024
 exit date: August 30th, 2024
 effective date: August 30th, 2024
 first date: null date
 next to last date: null date
 termination date: August 31st, 2024
 generation rule: Backward
 end of month: 0


,cusip,label,issue_date,maturity_date,time_to_maturity,high_investment_rate,int_rate,rank,outstanding,soma_holdings,stripping_amount,free_float,eod_yield,eod_price,accured,cbs_repriced_npv,cbs_repriced_ytm,cbs_price_spread,cbs_ytm_spread
0,912797LA3,"Sep 24s, 17-Week",2024-08-06,2024-09-03,0.013699,5.381000,NaN,16.0,0.000000e+00,6.784389e+08,0.000000e+00,-678.4389,8.185950,99.912833,0.000000,99.940883,5.484660,-0.028050,270.129012
1,912797LG0,"Sep 24s, 17-Week",2024-08-13,2024-09-10,0.032877,5.381000,NaN,15.0,0.000000e+00,6.873750e+08,0.000000e+00,-687.3750,6.475281,99.809694,0.000000,99.838340,5.456338,-0.028646,101.894268
2,912797LH8,"Sep 24s, 17-Week",2024-08-20,2024-09-17,0.052055,5.355000,NaN,14.0,0.000000e+00,7.210193e+08,0.000000e+00,-721.0193,6.061729,99.708333,0.000000,99.736941,5.428264,-0.028608,63.346553
3,912797LJ4,"Sep 24s, 17-Week",2024-08-27,2024-09-24,0.071233,5.335000,NaN,13.0,0.000000e+00,7.426366e+08,0.000000e+00,-742.6366,5.881169,99.607000,0.000000,99.636671,5.400441,-0.029671,48.072789
4,912797LK1,"Oct 24s, 17-Week",2024-08-06,2024-10-01,0.090411,5.346000,NaN,12.0,0.000000e+00,5.076268e+08,0.000000e+00,-507.6268,5.688607,99.510778,0.000000,99.537513,5.372870,-0.026735,31.573672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,912810TT5,"Aug 53s, 30-Year",2023-10-16,2053-08-15,28.980822,NaN,4.125,4.0,7.157430e+10,8.606596e+09,1.077172e+10,52195.9878,4.150915,99.562500,0.168139,99.569309,4.150783,0.161329,0.013202
386,912810TV0,"Nov 53s, 30-Year",2024-01-16,2053-11-15,29.232877,NaN,4.750,3.0,6.644364e+10,4.567153e+08,8.677531e+09,57309.3986,4.132526,110.406250,1.381114,110.292308,4.139850,1.495056,-0.732394
387,912810TX6,"Feb 54s, 30-Year",2024-04-15,2054-02-15,29.484932,NaN,4.250,2.0,7.119879e+10,2.211754e+09,1.127392e+10,57713.1192,4.142725,101.812500,0.173234,102.047749,4.129307,-0.062016,1.341796
388,912810UA4,"May 54s, 30-Year",2024-07-15,2054-05-15,29.728767,NaN,4.625,1.0,7.642080e+10,7.428279e+09,5.191628e+09,63800.8965,4.132281,108.375000,0.578125,108.632498,4.118931,0.320627,1.334967


In [45]:
fig = px.scatter(
    curve_set_df,
    x="time_to_maturity",
    y=f"{quote_type}_yield",
    color="original_security_term",
    hover_data=["label"],
    labels={"time_to_maturity": "Time to Maturity (Years)", "yield": "Yield (%)"},
    title="Filtered USTs Curve Set"
)

# fig.add_trace(
#     go.Scatter(
#         x=t2, y=fitted_zero_curve_func_ns(t2), mode="lines", name="Nelson-Siegel"
#     )
# )
# fig.add_trace(
#     go.Scatter(
#         x=t2, y=fitted_zero_curve_func_nss(t2), mode="lines", name="Svensson"
#     )
# )
# fig.add_trace(
#     go.Scatter(
#         x=t2, y=fitted_zero_curve_func_dl(t2), mode="lines", name="Diebold-Li"
#     )
# )
# fig.add_trace(
#     go.Scatter(
#         x=t2, y=fitted_zero_curve_func_bc(t2), mode="lines", name="Björk-Christensen"
#     )
# )
# fig.add_trace(
#     go.Scatter(
#         x=t2, y=fitted_zero_curve_func_aug_bc(t2), mode="lines", name="Augmented Björk-Christensen"
#     )
# )

# fig.add_trace(
#     go.Scatter(
#         x=t2, y=fitted_zero_curve_dict["ql_f_nss"]["zero_interp_func"](t2), mode="lines", name="Fitted NSS - Zero Curve"
#     )
# )
fig.add_trace(
    go.Scatter(
        x=t2, y=fitted_zero_curve_dict["ql_f_cbs"]["zero_interp_func"](t2), mode="lines", name="Fitted Zero Curve"
    )
)

fig.update_layout(
    legend_title_text='Label',
    xaxis_title="Time to Maturity (Years)",
    yaxis_title=f"YTMs: {quote_type}_yield",
    width=1600,
    height=800,
    template="plotly_dark",
)

fig.update_xaxes(
    showgrid=True,
    showspikes=True,
    spikecolor="white",
    spikesnap="cursor",
    spikemode="across",
)
fig.update_yaxes(showgrid=True, showspikes=True, spikecolor="white", spikethickness=1)
fig.show()